Sample code to parse out nested / complex JSON files

In [0]:
SourceURL = https://api.somedomain.com/api/v2/analytics/

token_type = "value"
access_token = "your access token"

request_headers = { "Authorization": token_type +' '+ access_token,"Content-Type": 'application/json'}

request_body ={
  "key1": "value1",
  "key2": "value2",
  "key3": "value3"}

response = requests.post(SourceURL,data=json.dumps(request_body),headers=request_headers,verify=False)
my_json=response.json()  
df=json_normalize(my_json['results'])

df_all=pd.DataFrame()
for i in range(len(df)):
    df_data_metrics=pd.DataFrame()  # initialization for each record in df
    
    df_head=df.iloc[i,:].to_frame().T
    df_head=df_head.drop(['data'],axis=1) # you can replace 'data' according to your dataset
    
    df_data=json_normalize(df['data'][i])   # you can replace 'data' according to your dataset
    
    for j in range(len(df_data)):
        df_data_t=df_data.iloc[j,:].to_frame().T           
        
        df_metrics=json_normalize(df_data['metrics'][j])        # you can replace 'metrics' according to your dataset    
     
        df_data_metrics_temp=pd.merge(df_data_t,df_metrics,how='cross')
        df_data_metrics_temp=df_data_metrics_temp.drop(['metrics'],axis=1)    # you can replace 'metrics' according to your dataset    
        df_data_metrics=pd.concat([df_data_metrics,df_data_metrics_temp])
        
    df_all_temp=pd.merge(df_head,df_data_metrics,how='cross')
    df_all=pd.concat([df_all,df_all_temp])
    
df_all.reset_index(drop=True,inplace=True)
df_all.columns=df_all.columns.str.upper()   